## A Notebook to get Information from existing IIIF-Manifests, select canvases, and arrange them into a new manifest


NOTE: For now, this will only work with existing manifests following the v2 of the IIIF Presentation API, but will produce a manifest following v3.

In [1]:
import requests
import json

In [14]:
# function to create a new manifest
def create_manifest(manifest_id, label, metadata, requiredStatement, provider):
   
    new_manifest = {
        "@context": "http://iiif.io/api/presentation/3/context.json",
        "id": manifest_id,
        "type": "Manifest",
        "label": label,
        "summary": requiredStatement,
        "metadata": metadata,
        "provider": provider,
        "items": []  # No canvases yet
    }  

    return new_manifest

In [15]:
# Provide basic Metadata for the new manifest
base_url="https://raw.githubusercontent.com/r0man-ist/iiif/refs/heads/main/"
filename ="test2.json"
manifest_id=base_url+filename ## required

label={"de": ["Rekonstruktion Yd7829"]} ## required
metadata=[
    {
         "label": "Autor",
         "value": ["Hensel, Fanny", "Hensel, Wilhelm"]
    },
    {
         "label": "Titel",
         "value": "Sammelband Yd7829"
    },
    {
      "label": "Signatur",
      "value": ["Yd7829"]
    },  
    {
      "label": "Sprache",
      "value": "ger"
    },
     {
      "label": "Lizenz",
      "value": "Public Domain Mark 1.0"
    },
        ]
requiredStatement = {
      "label": "Zu diesem Objekt",
      "value": "Bei diesem virtuellen Objekt handelt es sich um eine Rekonstruktion."
    },
provider = [
    {
      "id": "https://lab.sbb.berlin",
      "type": "Agent",
      "label": { "en": [ "Stabi Lab" ] },
      "homepage": [
        {
          "id": "https://lab.sbb.berlin",
          "type": "Text",
          "label": { "de": [ "Stabi Lab Homepage" ] },
          "format": "text/html"
        }]
    }      
  ]
    
new_manifest = create_manifest(manifest_id, label, metadata, requiredStatement, provider)

In [16]:
# function to get manifests from a list of urls
def fetch_manifests(manifest_urls):
    manifests = []
    for url in manifest_urls:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        manifest = response.json()  # Parse the JSON response
                             
        manifests.append(manifest)
            
    return manifests

In [17]:
# specify a list of manifests to fetch
manifest_urls = ["https://collections.library.yale.edu/manifests/17384131",
                 "https://collections.library.yale.edu/manifests/17384177",
                 "https://raw.githubusercontent.com/r0man-ist/iiif/refs/heads/main/manifests/bsb00062258v3.manifest.json",
                 "https://collections.library.yale.edu/manifests/17384255",
                 "https://raw.githubusercontent.com/r0man-ist/iiif/refs/heads/main/manifests/835033880v3.manifest.json"]
manifests = fetch_manifests(manifest_urls)
#print(manifests)



In [22]:
# get individual canvases from the manifests
canvases = []
for manifest in manifests:  
    for canvas in manifest.get('items', []):
        if canvas.get('type') == 'Canvas':
            canvases.append(canvas)

new_manifest['items'] = canvases

In [8]:
# OPTIONAL
# specify a list of canvases to REMOVE in the new manifest
to_exclude_canvas_list = [
    "https://content.staatsbibliothek-berlin.de/dc/1880377578-0235/canvas",
   "https://content.staatsbibliothek-berlin.de/dc/1880377578-0236/canvas",
   "https://content.staatsbibliothek-berlin.de/dc/1878144588-0159/canvas",
   "https://content.staatsbibliothek-berlin.de/dc/1880377578-0237/canvas",
   "https://content.staatsbibliothek-berlin.de/dc/1880377578-0238/canvas"
]


In [10]:
matching_canvases = []

# Create a dictionary for quick lookup using the canvas IDs
canvas_ids = {canvas["@id"]: canvas for canvas in canvases}

# Iterate over the canvas_list to maintain the specified order
for canvas_id in canvas_list:
    if canvas_id in to_exclude_canvas_list:
        del canvas_ids[canvas_id]

new_manifest["items"] = matching_canvases

NameError: name 'canvas_list' is not defined

In [23]:
new_manifest["items"] = canvases

In [24]:

with open("test2.json", "w", encoding="utf-8") as f:
        json.dump(new_manifest, f, ensure_ascii=False, indent=4)